In [2]:
# pandas 모듈을 pd라는 별칭으로 임포트
import pandas as pd

# konlpy 패키지에서 Okt 형태소 분석기를 임포트
from konlpy.tag import Okt

# CSV 파일 로드
file_path = './data/필터링된_스타벅스블로그본문.csv'  # 데이터가 저장된 CSV 파일 경로
data = pd.read_csv(file_path)  # CSV 파일을 읽어서 DataFrame으로 저장
# data = data.head(2)  # 데이터의 첫 2개 행만 선택

# 형태소 분석기 초기화
okt = Okt()  # Okt 객체를 생성하여 형태소 분석기 초기화

# 형태소 분석 및 명사 추출 함수
def extract_nouns(text):
    """
    입력된 텍스트에서 명사만 추출하는 함수
    :param text: 형태소 분석을 할 텍스트
    :return: 명사만 공백으로 구분하여 반환한 문자열
    """
    tokens = okt.pos(text)  # 텍스트를 형태소 단위로 토큰화하고 각 토큰의 품사를 태깅
    nouns = [word for word, pos in tokens if pos in ['Noun']]  # 태깅된 토큰 중 명사(Noun)만 추출
    return ' '.join(nouns)  # 추출된 명사를 공백으로 구분하여 하나의 문자열로 반환

# `Content` 컬럼에서 명사 추출
data['nouns'] = data['Content'].apply(extract_nouns)  # DataFrame의 'Content' 열에 대해 명사 추출 함수를 적용하여 새로운 'nouns' 열 생성

# 결과 저장
output_file_path = './data/스타벅스명사추출다시.csv'  # 결과를 저장할 CSV 파일 경로
data.to_csv(output_file_path, index=False)  # DataFrame을 CSV 파일로 저장, 인덱스는 저장하지 않음


In [2]:
# 결과 확인
data.drop(columns='Content')


,Store_Name,nouns
0,스타벅스 을지로4가역점,커피 디저트 여행 을지로 역 카페 스타벅스 을지로 역점 돈 산 후기 세상 여행 블로...
1,스타벅스 을지로경기빌딩점,냠냠냠 을지로 대형 카페 스타벅스 을지로 경기 빌딩 점 녜 스타벅스 이용 저 스타벅...


In [3]:
import pandas as pd
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from tokenization_morp import MecabTokenizer

# 데이터 로드
data_path = './csv/스타벅스블로그본문.csv'
df = pd.read_csv(data_path)
df = df.head(2)
device = torch.device("mps")

# 전처리 작업
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # 전화번호 제거
    text = re.sub(r'\d{2,3}-\d{3,4}-\d{4}', '', text)
    # 날짜 제거 (예: 2019.4.17 15:42)
    text = re.sub(r'\d{4}\.\d{1,2}\.\d{1,2} \d{1,2}:\d{2}', '', text)
    # 특수문자 제거
    text = re.sub(r'[!?,]', '', text)
    # 모든 이모티콘 제거
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
    # 기타 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)
    # 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_content'] = df['Content'].apply(clean_text)

# 형태소 분석기 설정
mecab_dic_path = '/Users/hyunbin/mecab-ko-dic-2.1.1-20180720'
mecab_tokenizer = MecabTokenizer(mecab_dic_path)

# KorBERT 모델 로드
config_path = 'bert_config.json'
model_bin_path = 'trained_bert_model.bin'  # 학습된 모델 파일 경로
vocab_file_path = 'vocab.korean_morp.list'

config = BertConfig.from_json_file(config_path)
bert_model = BertForSequenceClassification.from_pretrained(None, config=config, state_dict=torch.load(model_bin_path))
tokenizer = BertTokenizer(vocab_file=vocab_file_path, do_lower_case=False)

# 데이터셋 준비 함수
def prepare_dataset(texts, max_len=128):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = mecab_tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

texts = df['cleaned_content'].tolist()
input_ids, attention_masks = prepare_dataset(texts)

# 키워드 추출
def extract_keywords(input_ids, attention_masks, model):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(len(input_ids)):
            outputs = model(input_ids[i].unsqueeze(0), token_type_ids=None, attention_mask=attention_masks[i].unsqueeze(0))
            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).item()
            predictions.append(prediction)
    return predictions

predictions = extract_keywords(input_ids, attention_masks, bert_model)

# 키워드 결과를 DataFrame에 추가
df['keywords'] = predictions

# 매장명과 키워드만 추출
result_df = df[['Store_Name', 'keywords']]

# 결과 저장
result_df.to_csv('./csv/스타벅스명사추출테스트.csv', index=False)

ModuleNotFoundError: No module named 'tokenization_morp'

In [6]:
# pandas 모듈을 pd라는 별칭으로 임포트
import pandas as pd

# collections 모듈에서 Counter 클래스를 임포트
from collections import Counter

# scikit-learn 패키지에서 TfidfVectorizer를 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

# konlpy 패키지에서 Okt 형태소 분석기를 임포트
from konlpy.tag import Okt

# CSV 파일 로드
file_path = './data/스타벅스명사추출다시.csv'  # 데이터가 저장된 CSV 파일 경로
data = pd.read_csv(file_path)  # CSV 파일을 읽어서 DataFrame으로 저장

# 형태소 분석기 초기화
okt = Okt()  # Okt 객체를 생성하여 형태소 분석기 초기화

# 형태소 분석 및 명사 추출 함수
def extract_nouns(text):
    """
    입력된 텍스트에서 명사만 추출하는 함수
    :param text: 형태소 분석을 할 텍스트
    :return: 명사 리스트
    """
    tokens = okt.pos(text)  # 텍스트를 형태소 단위로 토큰화하고 각 토큰의 품사를 태깅
    nouns = [word for word, pos in tokens if pos in ['Noun']]  # 태깅된 토큰 중 명사(Noun)만 추출
    return nouns  # 명사 리스트 반환

# 불용어 목록 생성 함수
def generate_stopwords(nouns):
    """
    명사 리스트에서 빈도 상위 및 하위 1%의 단어를 불용어로 지정하는 함수
    :param nouns: 명사 리스트
    :return: 불용어 리스트
    """
    noun_counts = Counter(nouns)  # 명사 리스트에서 각 명사의 빈도수 계산
    total_nouns = len(noun_counts)  # 명사의 총 종류 수 계산
    top_1_percent = int(total_nouns * 0.20)  # 상위 1%에 해당하는 명사 수 계산
    bottom_1_percent = int(total_nouns * 0.20)  # 하위 1%에 해당하는 명사 수 계산
    
    # 상위 1% 빈도 명사와 하위 1% 빈도 명사를 불용어로 지정
    stopwords = [noun for noun, count in noun_counts.most_common(top_1_percent)]
    stopwords += [noun for noun, count in noun_counts.most_common()[:-bottom_1_percent-1:-1]]
    return stopwords  # 불용어 리스트 반환

# 불용어 제거된 명사 추출 함수
def filter_nouns(nouns, stopwords):
    """
    명사 리스트에서 불용어를 제거하는 함수
    :param nouns: 명사 리스트
    :param stopwords: 불용어 리스트
    :return: 불용어가 제거된 명사 문자열
    """
    return ' '.join([noun for noun in nouns if noun not in stopwords])  # 불용어를 제거한 명사들을 공백으로 구분하여 하나의 문자열로 반환

# 각 본문에 대해 독립적으로 명사 추출, 불용어 제거, TF-IDF 적용
store_names = []  # 매장 이름을 저장할 리스트 초기화
keywords = []  # 키워드를 저장할 리스트 초기화

# 각 행(블로그 글)에 대해 반복
for index, row in data.iterrows():
    content = row['Content']  # 블로그 글 내용
    store_name = row['Store_Name']  # 매장 이름
    
    # 명사 추출
    nouns = extract_nouns(content)
    
    # 불용어 목록 생성
    stopwords = generate_stopwords(nouns)
    
    # 불용어 제거
    filtered_nouns = filter_nouns(nouns, stopwords)
    
    # TF-IDF 적용
    vectorizer = TfidfVectorizer(max_features=100)  # 매장별로 상위 100개의 키워드 추출
    X = vectorizer.fit_transform([filtered_nouns])  # TF-IDF 행렬 생성
    
    # 키워드 추출
    tfidf_keywords = vectorizer.get_feature_names_out()  # TF-IDF 벡터에서 키워드 추출
    
    # 저장
    store_names.append(store_name)  # 매장 이름 리스트에 추가
    keywords.append(' '.join(tfidf_keywords))  # 키워드 리스트에 추가

# 결과 저장
result_df = pd.DataFrame({'Store_Name': store_names, 'Keywords': keywords})  # 매장 이름과 키워드를 DataFrame으로 저장
output_file_path = './csv/스타벅스키워드추천_결과테스트.csv'  # 결과를 저장할 CSV 파일 경로
result_df.to_csv(output_file_path, index=False)  # DataFrame을 CSV 파일로 저장, 인덱스는 저장하지 않음

# 결과 확인
result_df.head()  # 결과 DataFrame의 첫 5개 행 출력


,Store_Name,Keywords
0,스타벅스 을지로4가역점,가끔 가득 계란 공휴일 기본 때문 라이트 로비 매우 보고 보기 보늬밤 보시 보이시 ...
1,스타벅스 을지로경기빌딩점,가끔 거기 거리 걸음 결제 경영 고통 곳도 구경 구역 굿즈들 그거 근처 기기 네이버...
2,스타벅스 을지로국제빌딩점,가게 계속 고요 구역 노트북 리뷰 마감 마카롱 물가 뭔가 바로 별로 보고 분위기 블...
3,스타벅스 을지로삼화타워점,구경 그냥 근처 글레이 남대문로 녹차 느낌 는걸 대기업 대신 대한민국 던데 동시 먹...
4,스타벅스 을지로한국빌딩점,가게 가기 가야 가지 검사 고요 과정 나이 네이버 노트북 달라 대해 대형 디저트 라...


In [22]:
nouns = pd.read_csv('스타벅스_키워드_추천_결과.csv')
nouns

,Store_Name,Keywords
0,스타벅스 을지로4가역점,라떼 매장 메뉴 스벅 시간 음료 주말 중구 카페 트윈타워
1,스타벅스 을지로경기빌딩점,남대문로 매장 서울 시간 을지로입구역 제로 중구 카공 카페 평일
2,스타벅스 을지로국제빌딩점,라떼 매장 반납 방문 서울 세척 에코 이용 제로 중구
3,스타벅스 을지로삼화타워점,말차 사용 서울 시간 유저 을지로입구역 음료 주차 중구 카페
4,스타벅스 을지로한국빌딩점,국제 명동 반납 방문 생각 서울 을지로입구역 제로 주문 한국
...,...,...
1919,스타벅스 황학사거리점,메뉴 상가 서울 서울특별시 성동구 스벅 시간 주차 카드 커피
1920,스타벅스 황학캐슬점,거리 매장 서울특별시 아메리카노 주차 중구 청계천 카페 커피 학사
1921,스타벅스 회기역사거리점,동대문구 맛집 메뉴 브루 시간 에스프레소 음료 이문로 초콜릿 콜드
1922,스타벅스 회현역점,메뉴 사진 서울 아메리카노 주문 중구 카드 크리스마스 크림 퇴계로


In [1]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import HTMLResponse, FileResponse
from fastapi.templating import Jinja2Templates
from fastapi.requests import Request
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
from io import StringIO
import os

# Jupyter Notebook에서 이벤트 루프를 여러 번 실행할 수 있도록 설정
nest_asyncio.apply()

# FastAPI 애플리케이션 초기화
app = FastAPI()

# Jinja2 템플릿 설정
templates = Jinja2Templates(directory="templates")

# 형태소 분석기 초기화
okt = Okt()

# 형태소 분석 및 명사 추출 함수
def extract_nouns(text):
    tokens = okt.pos(text)
    nouns = [word for word, pos in tokens if pos in ['Noun']]
    return ' '.join(nouns)

# 불용어 목록 생성 함수
def generate_stopwords(nouns):
    noun_counts = Counter(nouns.split())
    total_nouns = len(noun_counts)
    top_1_percent = int(total_nouns * 0.20)
    bottom_1_percent = int(total_nouns * 0.20)
    
    stopwords = [noun for noun, count in noun_counts.most_common(top_1_percent)]
    stopwords += [noun for noun, count in noun_counts.most_common()[:-bottom_1_percent-1:-1]]
    return stopwords

# 불용어 제거된 명사 추출 함수
def filter_nouns(nouns, stopwords):
    return ' '.join([noun for noun in nouns.split() if noun not in stopwords])

@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.post("/extract-nouns/", response_class=HTMLResponse)
async def extract_nouns_endpoint(request: Request, file: UploadFile = File(...)):
    content = await file.read()
    data = pd.read_csv(StringIO(content.decode('utf-8')))
    data['nouns'] = data['Content'].apply(extract_nouns)
    result_data = data[['Store_Name', 'nouns']]
    
    # 결과 파일 저장
    output_file_path = './csv/스타벅스명사추출결과테스트.csv'
    result_data.to_csv(output_file_path, index=False)
    
    # nouns 축소 출력 및 '더보기' 기능
    result_data['nouns_short'] = result_data['nouns'].apply(lambda x: x[:100] + '...' if len(x) > 100 else x)
    
    result_html = result_data[['Store_Name', 'nouns_short']].to_html(escape=False, index=False)
    code_html = '''
    <pre>
    {code}
    </pre>
    '''.format(code='''
import pandas as pd
from konlpy.tag import Okt

# CSV 파일 로드
file_path = './csv/스타벅스블로그본문.csv'
data = pd.read_csv(file_path)
data = data.head(2)

# 형태소 분석기 초기화
okt = Okt()

# 형태소 분석 및 명사 추출 함수
def extract_nouns(text):
    tokens = okt.pos(text)
    nouns = [word for word, pos in tokens if pos in ['Noun']]
    return ' '.join(nouns)

# `Content` 컬럼에서 명사 추출
data['nouns'] = data['Content'].apply(extract_nouns)

# 결과 저장
output_file_path = './csv/스타벅스명사추출결과테스트.csv'
data.to_csv(output_file_path, index=False)
    '''.strip())
    
    return templates.TemplateResponse("result.html", {"request": request, "result": result_html, "code": code_html, "file_path": output_file_path})

@app.post("/generate-stopwords/", response_class=HTMLResponse)
async def generate_stopwords_endpoint(request: Request, file: UploadFile = File(...)):
    content = await file.read()
    data = pd.read_csv(StringIO(content.decode('utf-8')))
    
    all_nouns = ' '.join(data['nouns'])
    stopwords = generate_stopwords(all_nouns)
    data['filtered_nouns'] = data['nouns'].apply(lambda x: filter_nouns(x, stopwords))
    result_data = data[['Store_Name', 'filtered_nouns']]
    
    # 결과 파일 저장
    output_file_path = './csv/스타벅스키워드추천_결과테스트.csv'
    result_data.to_csv(output_file_path, index=False)
    
    # filtered_nouns 축소 출력 및 '더보기' 기능
    result_data['filtered_nouns_short'] = result_data['filtered_nouns'].apply(lambda x: x[:100] + '...' if len(x) > 100 else x)
    
    result_html = result_data[['Store_Name', 'filtered_nouns_short']].to_html(escape=False, index=False)
    code_html = '''
    <pre>
    {code}
    </pre>
    '''.format(code='''
import pandas as pd
from collections import Counter
from konlpy.tag import Okt

# CSV 파일 로드
file_path = './csv/스타벅스명사추출결과테스트.csv'
data = pd.read_csv(file_path)

# 형태소 분석기 초기화
okt = Okt()

# 형태소 분석 및 명사 추출 함수
def extract_nouns(text):
    tokens = okt.pos(text)
    nouns = [word for word, pos in tokens if pos in ['Noun']]
    return nouns

# 불용어 목록 생성 함수
def generate_stopwords(nouns):
    noun_counts = Counter(nouns)
    total_nouns = len(noun_counts)
    top_1_percent = int(total_nouns * 0.01)
    bottom_1_percent = int(total_nouns * 0.01)
    
    stopwords = [noun for noun, count in noun_counts.most_common(top_1_percent)]
    stopwords += [noun for noun, count in noun_counts.most_common()[:-bottom_1_percent-1:-1]]
    return stopwords

# 불용어 제거된 명사 추출 함수
def filter_nouns(nouns, stopwords):
    return ' '.join([noun for noun in nouns if noun not in stopwords])

data['filtered_nouns'] = data['nouns'].apply(lambda x: filter_nouns(x, stopwords))

# 결과 저장
output_file_path = './csv/스타벅스키워드추천_결과테스트.csv'
data.to_csv(output_file_path, index=False)
    '''.strip())
    
    return templates.TemplateResponse("result.html", {"request": request, "result": result_html, "code": code_html, "file_path": output_file_path})

@app.get("/download/")
async def download_file(file_path: str):
    return FileResponse(file_path, media_type='application/octet-stream', filename=os.path.basename(file_path))

# FastAPI 서버 실행
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [25443]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52487 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52994 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53008 - "POST /extract-nouns/ HTTP/1.1" 200 OK


/var/folders/wg/_dw7rbn14xl_4vkcr1327d040000gn/T/ipykernel_25443/469787622.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_data['nouns_short'] = result_data['nouns'].apply(lambda x: x[:100] + '...' if len(x) > 100 else x)


INFO:     127.0.0.1:53009 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53011 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53011 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:53011 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53011 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25443]


KeyboardInterrupt: 